<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Nicolás Cabello
- Nombre de alumno 2: Esteban Muñoz


### **Link de repositorio de GitHub:** `https://github.com/NicoCabello/Laboratorio_Progra_Cientifica`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 5.1 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.2 MB/s 
     |████████████████████████████████| 1.1 MB 37.8 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=9b36664f159c005c9fa1b5bbb670ef2bdf0d00693634cb0a621de5f0316015ae
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55509 sha256=7ce53241304aee7b9a2661fec8d5b9537098b85b003453f84c5d98c8b9afdb8d
  Stored in directory: /root/.cache/pip/wheels/19/bc/

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [2]:
def mount_drive(path):
    try:
        from google.colab import drive

        drive.mount("/content/drive")
        %cd {path}
    except: 
        print('Ignorando conexión drive-colab')

In [3]:
mount_drive("/content/drive/My Drive/MDS7202/Laboratorio 8")

Mounted at /content/drive
/content/drive/My Drive/MDS7202/Laboratorio 8


In [4]:
df_comics = pd.read_csv('df_comics.csv')
df_comics_no_label = pd.read_csv('comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [5]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [6]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [7]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['rock', 'best', 'actor', 'world'],
 ['new', 'york', 'beauti', 'citi']]

In [8]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [9]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,like,new,rock,roll,teacher,world,york
0,1,0,0,0,0,1,1,0,2,1,1,0,0
1,0,1,0,1,0,0,0,0,1,0,0,1,0
2,0,0,1,0,1,0,0,1,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [10]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor world,beauti,beauti citi,best,best actor,citi,good,...,rock,rock &,rock best,rock like,roll,teacher,teacher rock,world,york,york beauti
0,1,1,0,0,0,0,0,0,0,1,...,2,1,0,1,1,1,1,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,1,0,1,0,0,0,0,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [11]:
#### Código aquí ####
atributos_de_interes = [
    'intelligence_score',
    'strength_score',
    'speed_score',
    'durability_score',
    'power_score',
    'combat_score'
    ]

bow = CountVectorizer(
    tokenizer = StemmerTokenizer(),
    ngram_range = (1,2)
)

transformer = ColumnTransformer(
    transformers=[
        ('CountVectorizer', bow, 'history_text'),
        ('MinMaxScaler', MinMaxScaler(), atributos_de_interes)
    ]
)


## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [12]:
#### Código aquí ####
from sklearn.dummy import DummyClassifier

super_identifier_pipe = Pipeline([
    ('Preprocessing', transformer),
    ('Feature selection', SelectPercentile(f_classif, percentile=90)),
    ('Classifier', MultinomialNB())
])

X = df_comics.drop('alignment', axis=1)
y = df_comics.alignment

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

super_identifier_pipe.fit(X_train, y_train)
y_pred = super_identifier_pipe.predict(X_test)

print(classification_report(y_test, y_pred))

dummy = DummyClassifier(strategy='stratified', random_state=42)
dummy.fit(X_train, y_train)

y_pred_dummy = dummy.predict(X_test)

print(classification_report(y_test, y_pred_dummy))

              precision    recall  f1-score   support

         Bad       0.69      0.10      0.18        86
        Good       0.59      0.98      0.74       148
     Neutral       0.00      0.00      0.00        23

    accuracy                           0.60       257
   macro avg       0.43      0.36      0.31       257
weighted avg       0.57      0.60      0.49       257

              precision    recall  f1-score   support

         Bad       0.32      0.34      0.33        86
        Good       0.58      0.56      0.57       148
     Neutral       0.12      0.13      0.12        23

    accuracy                           0.45       257
   macro avg       0.34      0.34      0.34       257
weighted avg       0.46      0.45      0.45       257



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


El clasificador entrenado posee un muy buen *recall* para la clase *Good*, pero también cuenta con casi un 60% para la métrica *precision*. Esto quiere decir que existe un sesgo hacia esta clase, pues el modelo asigna esta etiqueta tanto para las muestras que realmente pertenecen a esta clase, como las de otras clases. Como consecuencia, el modelo no ha retornado para ningún ejemplo la clase *Neutral*, correspondiente a la clase con menos ejemplos. Además, sólo el 10% de las muestras con clase *Bad* se le han asignado correctamente esa clase.

El *DummyClassifier* si encuentra ejemplos de las clases *Neutral* y más ejemplos de la clase *Bad*, sin embargo, esto se debe a la forma en que se le ha pedido que haga las predicciones (stratified). Al comparar las *macro_avg* y las *weighted_avg* se puede notar que en practicamente todas el clasificador entrenado es mejor que el *DummyClassfier*, por lo que el primero es más confiable que el último.

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [40]:
#### Código aquí ####
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

params = [
    {
    'Preprocessing__CountVectorizer__ngram_range': [(1,1), (1,2), (1,3)],
    'Preprocessing__CountVectorizer__min_df': [1, 2],
    'Feature selection__percentile': [20, 40, 60, 80],
    'Classifier': [SVC()],
    'Classifier__C': [0.1, 1, 10]
    },

    {
    'Preprocessing__CountVectorizer__ngram_range': [(1,1), (1,2), (1,3)],
    'Preprocessing__CountVectorizer__min_df': [1, 2],
    'Feature selection__percentile': [20, 40, 60, 80],
    'Classifier': [RandomForestClassifier(random_state=42)],
    'Classifier__n_estimators': [100, 500],
    'Classifier__min_samples_split': [2, 3]
    },

    {
    'Preprocessing__CountVectorizer__ngram_range': [(1,1), (1,2), (1,3)],
    'Preprocessing__CountVectorizer__min_df': [1, 2],
    'Feature selection__percentile': [20, 40, 60, 80],
    'Classifier': [LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, random_state=42)],
    'Classifier__C': [0.1, 1, 10]
    }
]

search = HalvingGridSearchCV(
    super_identifier_pipe,
    params,
    cv=3,
    random_state=42,
    verbose=10).fit(X_train, y_train)

n_iterations: 4
n_required_iterations: 5
n_possible_iterations: 4
min_resources_: 18
max_resources_: 1028
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 240
n_resources: 18
Fitting 3 folds for each of 240 candidates, totalling 720 fits
[CV 1/3; 1/240] START Classifier=SVC(), Classifier__C=0.1, Feature selection__percentile=20, Preprocessing__CountVectorizer__min_df=1, Preprocessing__CountVectorizer__ngram_range=(1, 1)
[CV 1/3; 1/240] END Classifier=SVC(), Classifier__C=0.1, Feature selection__percentile=20, Preprocessing__CountVectorizer__min_df=1, Preprocessing__CountVectorizer__ngram_range=(1, 1);, score=(train=0.545, test=0.333) total time=   0.1s
[CV 2/3; 1/240] START Classifier=SVC(), Classifier__C=0.1, Feature selection__percentile=20, Preprocessing__CountVectorizer__min_df=1, Preprocessing__CountVectorizer__ngram_range=(1, 1)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


[CV 2/3; 1/240] END Classifier=SVC(), Classifier__C=0.1, Feature selection__percentile=20, Preprocessing__CountVectorizer__min_df=1, Preprocessing__CountVectorizer__ngram_range=(1, 1);, score=(train=0.545, test=0.333) total time=   0.2s
[CV 3/3; 1/240] START Classifier=SVC(), Classifier__C=0.1, Feature selection__percentile=20, Preprocessing__CountVectorizer__min_df=1, Preprocessing__CountVectorizer__ngram_range=(1, 1)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


[CV 3/3; 1/240] END Classifier=SVC(), Classifier__C=0.1, Feature selection__percentile=20, Preprocessing__CountVectorizer__min_df=1, Preprocessing__CountVectorizer__ngram_range=(1, 1);, score=(train=0.417, test=0.600) total time=   0.5s
[CV 1/3; 2/240] START Classifier=SVC(), Classifier__C=0.1, Feature selection__percentile=20, Preprocessing__CountVectorizer__min_df=1, Preprocessing__CountVectorizer__ngram_range=(1, 2)
[CV 1/3; 2/240] END Classifier=SVC(), Classifier__C=0.1, Feature selection__percentile=20, Preprocessing__CountVectorizer__min_df=1, Preprocessing__CountVectorizer__ngram_range=(1, 2);, score=(train=0.545, test=0.333) total time=   0.1s
[CV 2/3; 2/240] START Classifier=SVC(), Classifier__C=0.1, Feature selection__percentile=20, Preprocessing__CountVectorizer__min_df=1, Preprocessing__CountVectorizer__ngram_range=(1, 2)


...

[CV 1/3; 9/9] END Classifier=LogisticRegression(l1_ratio=0.5, penalty='elasticnet', random_state=42,
                   solver='saga'), Classifier__C=10, Feature selection__percentile=20, Preprocessing__CountVectorizer__min_df=2, Preprocessing__CountVectorizer__ngram_range=(1, 1);, score=(train=0.588, test=0.562) total time=  15.2s
[CV 2/3; 9/9] START Classifier=LogisticRegression(l1_ratio=0.5, penalty='elasticnet', random_state=42,
                   solver='saga'), Classifier__C=10, Feature selection__percentile=20, Preprocessing__CountVectorizer__min_df=2, Preprocessing__CountVectorizer__ngram_range=(1, 1)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


[CV 2/3; 9/9] END Classifier=LogisticRegression(l1_ratio=0.5, penalty='elasticnet', random_state=42,
                   solver='saga'), Classifier__C=10, Feature selection__percentile=20, Preprocessing__CountVectorizer__min_df=2, Preprocessing__CountVectorizer__ngram_range=(1, 1);, score=(train=0.666, test=0.543) total time=   9.1s
[CV 3/3; 9/9] START Classifier=LogisticRegression(l1_ratio=0.5, penalty='elasticnet', random_state=42,
                   solver='saga'), Classifier__C=10, Feature selection__percentile=20, Preprocessing__CountVectorizer__min_df=2, Preprocessing__CountVectorizer__ngram_range=(1, 1)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


[CV 3/3; 9/9] END Classifier=LogisticRegression(l1_ratio=0.5, penalty='elasticnet', random_state=42,
                   solver='saga'), Classifier__C=10, Feature selection__percentile=20, Preprocessing__CountVectorizer__min_df=2, Preprocessing__CountVectorizer__ngram_range=(1, 1);, score=(train=0.577, test=0.596) total time=   9.3s


In [41]:
search.best_params_

{'Classifier': RandomForestClassifier(random_state=42),
 'Classifier__min_samples_split': 2,
 'Classifier__n_estimators': 100,
 'Feature selection__percentile': 60,
 'Preprocessing__CountVectorizer__min_df': 2,
 'Preprocessing__CountVectorizer__ngram_range': (1, 1)}

<!-- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch. -->

```
La búsqueda entregó los siguientes parámetros como los mejores:
- Clasificador: Random Forest
- min_sample_split: 2
- n_estimators: 100
- percentile: 60
- min_df: 2
- ngram_range: (1,1)


El clasificador SVM necesita encontrar vectores de soporte para realizar sus predicciones, pero ante la alta dimensionalidad del problema no logra ajustar los vectores de soporte mejor que RandomForest.
Adicionalmente el resultado de Support Vector Machine se ve influenciado por el factor de regularización C, el cual no fue ajustado con mucho detalle en la ejecución de este código.

Por parte de Random Forest, el caracter aleatorio pudo facilitar la obtención de mejores resultados, mitigando un poco el efecto de no ajustar muchos parámetros de forma exhaustiva.

En cuanto al clasificador de regresión logística

Por parte de la regression logística el tipo de penalización, solver y demás parámetros a utilizar también afectan al problema.
Por ende puede que al no seleccionarlos de forma cuidadosa, el rendimiento haya sido inferior a Random Forest.

El percentil de características utilizado llegó a 60, posiblemente porque 80 serían demasiadas características para ajustarse y 40 o 20 muy pocas para equipaprarse al nivel de predicción obtenido con 60.

El rango de n-gramas se mantuvo en 1, siendo una razón posible que al aumentar los n-gramas se generan demasiadas características y los datos no son suficientes para capturar su comportamiento (undefitting).

En cuanto al min_df se determina que entre 1 y 2, el umbral necesario para que sea considerada como caractterística es 1.
Posiblemente por la naturaleza del problema se detecten palabras individuales asociadas a comportamientos buenos, neutrales o malignos, sin necesitar palabras compuestas.
También debido al uso del percetil de características se redujo la cantidad de palabras individuales que no aportaban, y como son menos que las de un bigrama no se necesitan tantos datos para generar una buena predicción.


```

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [54]:
#### Código aquí ####
cleaned_comics_no_label = df_comics_no_label.dropna(subset=["history_text"])
to_predict = cleaned_comics_no_label.loc[cleaned_comics_no_label["name"].isin(["Vergil", "Gorilla Girl", "Batcow"])]
to_predict

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
16,122,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,529,Gorilla Girl,Fahnbullah Eddy,Fahnbullah Eddy,7,A carnival performer with the ability to turn ...,Gorilla Girl can transform into a talking gori...,90,35,60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
78,1368,Vergil,Vergil Sparda,NaN,16,"Vergil, later also known as Nelo Angelo, is on...",NaN,90,75,95,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
83,122,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
search.predict(to_predict)

array(['Good', 'Good', 'Good', 'Good'], dtype=object)

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>